
## Class Objects Definition<br>

In [ ]:
import random

suits = ('Hearts', 'Diamonds', 'Spades', 'Clubs')
ranks = ('Two', 'Three', 'Four', 'Five', 'Six', 'Seven', 'Eight', 'Nine', 'Ten', 'Jack', 'Queen', 'King', 'Ace')
values = {'Two': 2, 'Three': 3, 'Four': 4, 'Five': 5, 'Six': 6, 'Seven': 7, 'Eight': 8, 'Nine': 9, 'Ten': 10,
          'Jack': 10, 'Queen': 10, 'King': 10, 'Ace': 11}


class Card:
    def __init__(self, suit, rank):
        self.suit = suit
        self.rank = rank
        self.value = values[rank]

    def __str__(self):
        return f'{self.rank} of {self.suit}'


In [ ]:
class Deck:
    def __init__(self):
        self.deck = []
        for rank in ranks:
            for suit in suits:
                card = Card(suit, rank)
                self.deck.append(card)

    def __str__(self):
        deck_comp = ''
        for card in self.deck:
            deck_comp + '\n' + card.__str__()
        return deck_comp

    def shuffle(self):
        random.shuffle(self.deck)

    def deal(self):
        cards = []
        for _ in range(2):
            cards.append(self.deck.pop())
        return cards

    def hit(self):
        return self.deck.pop()

    def reset(self):
        self.__init__()
        

In [ ]:
class Hand:
    def __init__(self):
        self.cards = []
        self.value = 0
        self.aces = 0

    def add_card(self, card):
        self.cards.append(card)
        self.value += values[card.rank]
        for card in self.cards:
            if card.rank == 'Ace':
                self.aces += 1

    def __str__(self):
        card_comp = ''
        for card in self.cards:
            card_comp += '\n' + card.__str__()
        return card_comp

    def adjust_for_ace(self):
        while self.value > 21 and self.aces:
            self.value -= 10
            self.aces -= 1

    def reset(self):
        self.__init__()
        

In [ ]:
class Chips:
    def __init__(self):
        self.total = 100  # This can be set to a default value or supplied by a user input
        self.bet = 0

    def place_bet(self, bet):
        self.bet += bet

    def win_bet(self):
        self.total += self.bet

    def lose_bet(self):
        self.total -= self.bet
        

## Blackjack Game Function Definitions

### Initial Setup

In [ ]:
deck_of_cards = Deck()
deck_of_cards.shuffle()
user = Hand()
computer = Hand()
chips = Chips()


def take_bet():
    try:
        bet = int(input('Please enter your bet in number: '))
    except ValueError:
        print('Please make sure to enter a valid number.')
        take_bet()
    else:
        chips.place_bet(bet)


def new_game_setup():
    print('\nWelcome to Blackjack! \n')
    take_bet()
    for _ in range(2):
        dealt_cards = deck_of_cards.deal()
        user.add_card(dealt_cards[0])
        computer.add_card(dealt_cards[1])
    print(f'Computer has {computer.cards[-1]} and one more card')
    print(f'You have {user.cards[0]} and {user.cards[1]}')
    print(f'Your cards\' sum is: {user.value}')
    start_playing()
    

### Game logics

In [ ]:
def start_playing():
    while True:
        if check_for_blackjack():
            break
        user.adjust_for_ace()
        computer.adjust_for_ace()
        user_choice = input('Enter h for hit or s for stand: ')
        if user_choice == 'h':
            hit_or_stand('hit', user)
            if user.value > 21:
                find_out_the_winner()
                break
        else:
            hit_or_stand('stand', user)
            while computer.value < 17:
                hit_or_stand('hit', computer)
            else:
                hit_or_stand('stand', computer)
                break


def hit_or_stand(choice, player):
    match choice:
        case 'hit':
            player.add_card(deck_of_cards.hit())
            if player == user:
                print(f'Your cards are: \n{user}')
                print(f'Your new total is: {player.value}')
        case 'stand':
            if player == computer:
                print(f'Computer\'s cards are: \n{computer}')
                print(f'Computer\'s new total is: {player.value}')
                find_out_the_winner()
                

### Finding out the Winner

In [ ]:
def check_for_blackjack():
    if computer.value == 21:
        print(f'Computer wins with a blackjack: {computer.value}')
        chips.lose_bet()
        print(f'Total chips => {chips.total}')
        reset()
        return True
    elif user.value == 21:
        print(f'You win with a blackjack: {user.value}')
        chips.win_bet()
        print(f'Total chips => {chips.total}')
        reset()
        return True


def find_out_the_winner():
    if computer.value > 21:
        print(f'Computer busts. It has a total of {computer.value} You win!')
        chips.win_bet()
        print(f'Total chips => {chips.total}')
    elif user.value > 21:
        print(f'Computer wins! You went over 21. Your total was {user.value}')
        chips.lose_bet()
        print(f'Total chips => {chips.total}')
    elif computer.value > user.value:
        print(f'Computer wins {computer.value} to {user.value}')
        chips.lose_bet()
        print(f'Total chips => {chips.total}')
    elif user.value > computer.value:
        print(f'You win {user.value} to {computer.value}')
        chips.win_bet()
        print(f'Total chips => {chips.total}')
    else:
        print('It is a tie!')
    reset()
    

### Resetting and Restarting the Game

In [ ]:
def reset():
    play_again = input('Would you like to play again? (y/n): ')
    if play_again == 'y':
        user.reset()
        computer.reset()
        chips.bet = 0
        deck_of_cards.reset()
        deck_of_cards.shuffle()
        new_game_setup()

new_game_setup()